In [1]:
import os

In [2]:
!tree ../dataset/Сокращение\ по\ частям\ речи

../dataset/Сокращение по частям речи
├── 1.Первый жанр исходная выборка.txt
├── 2.Первый жанр без клауз, включающих наречия.txt
├── 3.Первый жанр без клауз, включающих глаголы.txt
├── 4. Первый жанр без клауз, включающих глаголы и наречия.txt
├── Без прилагательных второй жанр.txt
├── Без прилагательных первый жанр.txt
└── Случайные выборки.txt

0 directories, 7 files


In [3]:
from nltk.tokenize import word_tokenize

In [4]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab') # Descarga los datos para la correcta tokenización de los idiomas.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
import re
#from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize

def segment_text_with_ngrams(text, n=3):  
    """Segmenta el texto dividiéndolo en frases, evitando dividir dentro de paréntesis."""
    
    def replace_inside_parentheses(match):
        """Reemplaza los signos de puntuación dentro de los paréntesis para evitar segmentación incorrecta."""
        content = match.group(0)
        content = content.replace(".", "§").replace("!", "§").replace("?", "§")
        return content  # Devuelve el contenido modificado
    
    text = re.sub(r'\([^()]*\)', replace_inside_parentheses, text)  # Evita problemas con paréntesis anidados

    sentences = sent_tokenize(text, language="russian")  # Segmentar en frases correctamente

    segmented_sentences = []
    for sentence in sentences:
        sentence = sentence.replace("§", ".")  # Restaurar puntuación dentro de paréntesis
        words = word_tokenize(sentence, language="russian")  # Tokenización de palabras
        if words:
            segmented_sentences.append(" ".join(words))

    return [s.strip() for s in segmented_sentences if s.strip()]

def clean_text(text):
    """Elimina texto entre corchetes y caracteres innecesarios."""
    cleaned_text = re.sub(r'\[.*?\]', '', text)  # Eliminar contenido entre corchetes []
    return cleaned_text.strip()

def load_texts(filename, label):
    with open(filename, "r", encoding="utf-8") as f:
        text = f.read()
    
    sentences = segment_text_with_ngrams(text)  
    cleaned_sentences = [clean_text(sentence) for sentence in sentences]
    
    return [{"text": sentence, "label": label} for sentence in cleaned_sentences if sentence]

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar ambos géneros
genre1 = load_texts("../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt", label="0")
genre2 = load_texts("../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt", label="1")

# Combinar ambos géneros en un DataFrame
all_texts_df = pd.DataFrame(genre1 + genre2)

# Eliminar filas con valores NaN o vacíos en la columna 'text'
all_texts_df = all_texts_df[all_texts_df['text'].notna()]
all_texts_df = all_texts_df[all_texts_df['text'] != '']

# Dividir nuevamente en train, validation y test
train_df, test_df = train_test_split(all_texts_df, test_size=0.2, random_state=42, stratify=all_texts_df['label'])

# Guardar los conjuntos limpios
all_texts_df.to_csv("data_without_adjectives.csv",index=False)
train_df.to_csv("train_without_adjectives.csv", index=False)
test_df.to_csv("test_without_adjectives.csv", index=False)

print(f"Tamaño del conjunto completo: {len(all_texts_df)}")
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba: {len(test_df)}")

Tamaño del conjunto completo: 880
Tamaño del conjunto de entrenamiento: 704
Tamaño del conjunto de prueba: 176


In [7]:
import pandas as pd

# Cargar los datos desde los archivos CSV
all_texts_df = pd.read_csv("data_without_adjectives.csv")
train_df = pd.read_csv("train_without_adjectives.csv")
test_df = pd.read_csv("test_without_adjectives.csv")

# Mostrar las primeras filas de cada DataFrame
print("Datos completos:")
print(all_texts_df.head(), len(all_texts_df))

print("\nConjunto de entrenamiento:")
print(train_df.head(),len(train_df))

print("\nConjunto de prueba:")
print(test_df.head(),len(test_df))



Datos completos:
                                                text  label
0                                  Кустарник Ветви .      0
1  побеги при сушке Почки со слегка уплощенным но...      0
2           Прилистники , по краю , Черешки сверху .      0
3  Листья , в основании с зубцами или почти , сле...      0
4             побеги с 2-6 листьями ; оси соцветий .      0 880

Conjunto de entrenamiento:
                                                text  label
0                                          Щетинки .      0
1  механизм адаптации целостного растения к услов...      1
2           Кисти 3-4.5 см дл. , цветки поникающие .      0
3  ориентировка : Ng = а , Nm = b. В свете минера...      0
4  Листья на черешках 2-3 см дл. ; с влагалищами ...      0 704

Conjunto de prueba:
                                                text  label
0      спайность наблюдается в направлении ( 001 ) .      0
1  Метатарзус лапок сужен в основании , его ширин...      0
2  побеги при сушке Почки 

In [8]:
# Cargar los conjuntos de datos
train_df = pd.read_csv('train_without_adjectives.csv')
test_df = pd.read_csv('../Step_1/data.csv')

In [9]:
from nltk.util import ngrams
from nltk import word_tokenize

# Función para generar ngramas
def generate_ngrams(text, n=3):
    tokens = word_tokenize(text.lower())  # Convertir el texto a minúsculas y tokenizar
    return set(ngrams(tokens, n))  # Generar ngramas y devolverlos como un conjunto

# Comparar ngramas entre conjuntos
def find_similar_samples(train_texts, test_texts, n=3):
    similar_indices = []
    # Generar ngramas para train y val
    train_ngrams = [generate_ngrams(text, n) for text in train_texts]    
    # Comparar con los textos de test
    for i, test_text in enumerate(test_texts):
        test_ngrams = generate_ngrams(test_text, n)
        
        for train_ngram in train_ngrams:
            if test_ngrams & train_ngram:  # Verificar si hay intersección
                similar_indices.append(i)
                break
    
    return similar_indices


In [10]:
train_texts = train_df['text'].astype(str).tolist()
test_texts = test_df['text'].astype(str).tolist()

similar_indices = find_similar_samples(train_texts, test_texts, n=3)
test_df = test_df.drop(similar_indices)


In [11]:
#train_df.to_csv("train_without_adjectives.csv", index=False)
#val_df.to_csv("val_without_adjectives.csv", index=False)
test_df.to_csv("test_orginal.csv", index=False)

In [12]:
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de prueba(original): {len(test_df)}")

Tamaño del conjunto de entrenamiento: 704
Tamaño del conjunto de prueba(original): 193
